# External Test: PhantomX Dataset

Bu notebook, eğitim verisinden tamamen bağımsız olan PhantomX datasetinde modeli test eder.

**PhantomX Dataset:**
- 2 phantom (D55-01, D55-02)
- Her biri için düşük doz (40 mAs) ve yüksek doz (300 mAs) görüntüler
- 3 farklı rekonstrüksiyon: FBP, AIDR3D, AiCE

---

## İş Akışı

1. PhantomX DICOM → NPY dönüşümü
2. Model ile denoising
3. PSNR/SSIM hesaplama
4. Görsel karşılaştırma

In [ ]:
!pip install pydicom -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. Model'i Yükle

In [ ]:
%run 01_model_architecture.ipynb

In [ ]:
import os
import glob

checkpoint_dir = "/content/drive/MyDrive/Tasarım Dersi/Projects/model_checkpoints"

list_of_files = glob.glob(os.path.join(checkpoint_dir, 'G_epoch_*.h5'))
latest_file = max(list_of_files, key=os.path.getctime)
print(f"Yüklenen Model: {os.path.basename(latest_file)}")

generator = build_generator()
generator.load_weights(latest_file)
print("✅ Model hazır.")

## 2. PhantomX DICOM → NPY Dönüşümü

Mayo dataset ile aynı preprocessing adımlarını uyguluyoruz:
- HU dönüşümü
- [-1000, 1000] clip
- [-1, 1] normalizasyon

In [ ]:
import numpy as np
import pydicom

# Ayarlar
phantomx_base_dir = "/content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/phantomx_abdomen_pelvis_dataset"
output_dir = "/content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/Output"

phantoms = ["D55-01", "D55-02"]

print("="*80)
print("PhantomX DICOM → NPY Dönüştürücü")
print("="*80)

In [ ]:
def dicom_klasoru_oku(klasor_yolu, klasor_adi):
    """DICOM dosyalarını okur ve 3D numpy array olarak döndürür"""
    print(f"      📂 {klasor_adi}")

    dicom_dosyalari = []

    for dosya in os.listdir(klasor_yolu):
        if dosya.endswith('.dcm'):
            dosya_yolu = os.path.join(klasor_yolu, dosya)
            dicom_dosyalari.append(dosya_yolu)

    if not dicom_dosyalari:
        print(f"      ❌ DICOM dosyası bulunamadı!")
        return None

    print(f"      ✓ {len(dicom_dosyalari)} DICOM dosyası bulundu")

    # DICOM dosyalarını oku ve sırala
    dicom_veriler = []

    for dosya in sorted(dicom_dosyalari):
        try:
            ds = pydicom.dcmread(dosya)

            pixel_array = ds.pixel_array.astype(np.float32)

            # HU Dönüşümü
            intercept = ds.RescaleIntercept if 'RescaleIntercept' in ds else 0
            slope = ds.RescaleSlope if 'RescaleSlope' in ds else 1
            pixel_array = pixel_array * slope + intercept

            # Clip
            pixel_array = np.clip(pixel_array, -1000, 1000)

            # Z koordinatı
            if hasattr(ds, 'ImagePositionPatient'):
                slice_location = float(ds.ImagePositionPatient[2])
            elif hasattr(ds, 'SliceLocation'):
                slice_location = float(ds.SliceLocation)
            elif hasattr(ds, 'InstanceNumber'):
                slice_location = float(ds.InstanceNumber)
            else:
                slice_location = len(dicom_veriler)

            dicom_veriler.append((slice_location, pixel_array))

        except Exception as e:
            continue

    # Z koordinatına göre sırala
    dicom_veriler.sort(key=lambda x: x[0])

    # 3D array oluştur
    volume_3d = np.stack([slice_data for _, slice_data in dicom_veriler])

    print(f"      ✓ Volume: {volume_3d.shape}")

    return volume_3d


def normalizasyon(volume, min_hu=-1000, max_hu=1000):
    """HU değerlerini [-1, 1] aralığına normalize eder"""
    volume = np.clip(volume, min_hu, max_hu)
    volume = (volume - min_hu) / (max_hu - min_hu)  # [0, 1]
    volume = volume * 2 - 1  # [-1, 1]
    return volume.astype(np.float32)

## 3. Phantom Test

Her phantom için:
1. Düşük doz görüntüleri al
2. Model ile iyileştir
3. Yüksek doz referans ile karşılaştır

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

# FBP rekonstrüksiyon kullanıyoruz (model bununla eğitildi)
rekon_secim = "FBP"

for phantom_name in phantoms:
    print(f"\n{'='*80}")
    print(f"🔬 Phantom: {phantom_name}")
    print('='*80)
    
    phantom_path = os.path.join(phantomx_base_dir, phantom_name)
    
    # Low dose (40) ve High dose (300) klasörlerini bul
    low_dose_path = None
    high_dose_path = None
    
    for dose_folder in os.listdir(phantom_path):
        dose_path = os.path.join(phantom_path, dose_folder)
        if not os.path.isdir(dose_path):
            continue
        
        # FBP klasörünü bul
        for rekon_folder in os.listdir(dose_path):
            if rekon_secim.upper() in rekon_folder.upper():
                if dose_folder == "40":
                    low_dose_path = os.path.join(dose_path, rekon_folder)
                elif dose_folder == "300":
                    high_dose_path = os.path.join(dose_path, rekon_folder)
    
    if not low_dose_path or not high_dose_path:
        print(f"  ❌ Klasörler bulunamadı!")
        continue
    
    # DICOM oku
    print(f"\n  📥 Low Dose okuyuyor...")
    low_volume = dicom_klasoru_oku(low_dose_path, os.path.basename(low_dose_path))
    
    print(f"\n  📤 High Dose okuyuyor...")
    high_volume = dicom_klasoru_oku(high_dose_path, os.path.basename(high_dose_path))
    
    if low_volume is None or high_volume is None:
        continue
    
    # Normalizasyon
    low_norm = normalizasyon(low_volume)
    high_norm = normalizasyon(high_volume)
    
    # Resize (model 256x256 bekliyor)
    low_resized = np.array([cv2.resize(s, (256, 256)) for s in low_norm])
    high_resized = np.array([cv2.resize(s, (256, 256)) for s in high_norm])
    
    # Channel ekle
    low_input = low_resized[..., np.newaxis]
    high_target = high_resized[..., np.newaxis]
    
    # Inference
    print(f"\n  🔄 Model çalıştırılıyor...")
    predictions = generator(low_input, training=False)
    
    # PSNR/SSIM
    psnr = tf.image.psnr(high_target, predictions, max_val=2.0).numpy()
    ssim = tf.image.ssim(high_target, predictions, max_val=2.0).numpy()
    
    print(f"\n  📊 Sonuçlar:")
    print(f"     PSNR: {np.mean(psnr):.4f} dB (±{np.std(psnr):.4f})")
    print(f"     SSIM: {np.mean(ssim):.4f} (±{np.std(ssim):.4f})")
    
    # Örnek görsel
    mid_slice = len(low_input) // 2
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(low_input[mid_slice, :, :, 0], cmap='gray')
    axes[0].set_title('Low Dose (Input)')
    axes[0].axis('off')
    
    axes[1].imshow(predictions[mid_slice, :, :, 0], cmap='gray')
    axes[1].set_title('AI Enhanced')
    axes[1].axis('off')
    
    axes[2].imshow(high_target[mid_slice, :, :, 0], cmap='gray')
    axes[2].set_title('High Dose (Reference)')
    axes[2].axis('off')
    
    plt.suptitle(f'{phantom_name} - Slice {mid_slice}', fontsize=14)
    plt.tight_layout()
    plt.savefig(f'{phantom_name}_comparison.png', dpi=150)
    plt.show()